### Importing libraries

In [1]:
!pip install ipynb

In [2]:
import logging
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
import numpy as np
import time
import os
import sys
from ipynb.fs.defs.SiameseModel import Recognizer
import logging
import tensorflow as tf

### Image Dimensions

In [3]:
data_dimension = 128

### Input and target vectors

In [4]:
# Loading input and target vectors

X1 = np.load('x1.npy')
X2 = np.load('x2.npy')
Y = np.load('y.npy')

# Reshaping the target vectors
X1 = X1.reshape( ( X1.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )
X2 = X2.reshape( ( X2.shape[0]  , data_dimension**2 * 3  ) ).astype( np.float32 )

print( X1.shape )
print( X2.shape )
print( Y.shape )

(400, 49152)
(400, 49152)
(400,)


### Training and saving the model

In [5]:
# Creating an instance of siamese model
recognizer = Recognizer()

In [6]:
# Defining hyper parameters
parameters = {
    'batch_size' : 6 ,
    'epochs' : 5 ,
    'callbacks' : None , # [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
    'val_data' : None
}

# Training the siamese model
recognizer.fit( [ X1 , X2 ], Y, hyperparameters=parameters)

67/67 [==============================] - 79s 1s/step - loss: 0.4891
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 49152)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 49152)]      0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 64)           54243072    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)    

In [7]:
# saving the model
recognizer.save_model('model.h5')

### Loading images from directory

In [8]:
# Enter the path name untill 'images' [ Actual images are stored at ...images/sanyam/image0.png]
# Here enter the path till the local images
# Custom images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\Test_Images\
# p1 : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Chetana\
# p2 : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Prabhleen\
# p3 : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Sanyam\
# p4 :  C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Taruneesh\
dir_path_custom_images = input( 'Enter images directory path for custom images : ')
dir_path_class1_images = input( 'Enter images directory path for first class images : ')
dir_path_class2_images = input( 'Enter images directory path for second class images : ')
dir_path_class3_images = input( 'Enter images directory path for second class images : ')
dir_path_class4_images = input( 'Enter images directory path for second class images : ')




Enter images directory path for custom images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\Test_Images\
Enter images directory path for first class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Chetana\
Enter images directory path for second class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Prabhleen\
Enter images directory path for second class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Sanyam\
Enter images directory path for second class images : C:\Users\sanya\Desktop\Calgary\ENEL645\Siamese_Face_Project\Enel645_FaceDetection_SiameseModel\images_dataset\Taruneesh\


In [9]:
# storing the numpy arrays of all images
custom_images = recognizer.prepare_images_from_dir( dir_path_custom_images  )
class_1_images = recognizer.prepare_images_from_dir( dir_path_class1_images )
class_2_images = recognizer.prepare_images_from_dir( dir_path_class2_images )
class_3_images = recognizer.prepare_images_from_dir( dir_path_class3_images )
class_4_images = recognizer.prepare_images_from_dir( dir_path_class4_images )

In [10]:
# printing shape of all the images
print("The shape of the numpy array of class first images", class_1_images.shape)
print("The shape of the numpy array of class second images", class_2_images.shape)
print("The shape of the numpy array of class second images", class_3_images.shape)
print("The shape of the numpy array of class second images", class_4_images.shape)

print("The shape of the numpy array of custom images", custom_images.shape)

The shape of the numpy array of class first images (5, 49152)
The shape of the numpy array of class second images (5, 49152)
The shape of the numpy array of class second images (5, 49152)
The shape of the numpy array of class second images (5, 49152)
The shape of the numpy array of custom images (8, 49152)


### Classifying images

In [11]:
# Predicting the score for each image

# scores stores a list of scores for each custom image with the train dataset
scores = list()

# labels is a list of labels for each custom image against the class label of trained image
labels = list()

# looping through all custom images
for image in custom_images:
    label = list()
    score = list()
    # looping across images of class 1
    for sample in class_1_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 0 to this class
        label.append( 0 )
    # looping across images of class 2
    for sample in class_2_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 1 for this class
        label.append( 1 )
    # looping across images of class 3
    for sample in class_3_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 1 for this class
        label.append( 2 )
    # looping across images of class 4
    for sample in class_4_images :
        image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
        score.append( recognizer.predict( [ image , sample ])[0] )
        # appending label 1 for this class
        label.append( 3 )
#     # looping across images of class 5
#     for sample in class_5_images :
#         image , sample = image.reshape( ( 1 , -1 ) ) , sample.reshape((1 , -1 ) )
#         score.append( recognizer.predict( [ image , sample ])[0] )
#         # appending label 1 for this class
#         label.append( 4 )
    labels.append( label )
    scores.append( score )

### Analysing the scores and labels

In [12]:
print("The scores are: ", scores)
print("The labels are: ", labels)

scores = np.array( scores )
labels = np.array( labels )

print('The shape of the numpy array of labels are: ', scores.shape)
print('The shape of the numpy array of scores are:', labels.shape )
print('The scores are: ', scores)
print('The lables are: ', labels)

The scores are:  [[array([0.36300611], dtype=float32), array([0.36300611], dtype=float32), array([0.43088087], dtype=float32), array([0.48405963], dtype=float32), array([0.38549522], dtype=float32), array([0.26712358], dtype=float32), array([0.2863428], dtype=float32), array([0.26712358], dtype=float32), array([0.26712358], dtype=float32), array([0.2863428], dtype=float32), array([0.37838963], dtype=float32), array([0.37838963], dtype=float32), array([0.37838963], dtype=float32), array([0.37838963], dtype=float32), array([0.37838963], dtype=float32), array([0.2268976], dtype=float32), array([0.2268976], dtype=float32), array([0.23803928], dtype=float32), array([0.24418685], dtype=float32), array([0.2268976], dtype=float32)], [array([0.17776689], dtype=float32), array([0.17776689], dtype=float32), array([0.22313863], dtype=float32), array([0.22704044], dtype=float32), array([0.16415888], dtype=float32), array([0.46099502], dtype=float32), array([0.4372353], dtype=float32), array([0.4609

### Classifying into classes based on best score

In [13]:
for i in range( custom_images.shape[0] ) :
    index = np.argmax( scores[i] )
    label_ = labels[i][index]
    print( 'IMAGE {} is {} with confidence of {}'.format( i+1  , label_ , scores[i][index][0] ) )

IMAGE 1 is 0 with confidence of 0.4840596318244934
IMAGE 2 is 3 with confidence of 0.4994640350341797
IMAGE 3 is 3 with confidence of 0.46890583634376526
IMAGE 4 is 1 with confidence of 0.4970676004886627
IMAGE 5 is 3 with confidence of 0.46890583634376526
IMAGE 6 is 2 with confidence of 0.4928792715072632
IMAGE 7 is 2 with confidence of 0.4055774211883545
IMAGE 8 is 3 with confidence of 0.44377219676971436
